# Plugins

> Optional plugin integration for extensible settings systems

In [ ]:
#| default_exp plugins

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
from dataclasses import dataclass
from pathlib import Path
from typing import Dict, Any, Optional, Protocol, runtime_checkable

## Plugin Metadata

In [ ]:
#| export
# Note: No predefined PluginCategory enum - applications define their own categories as strings
# Examples: "transcription", "export", "analysis", "integration", etc.

In [ ]:
#| export
@dataclass
class PluginMetadata:
    """Metadata describing a plugin.
    
    This dataclass holds information about a plugin that can be displayed
    in the settings UI without loading the actual plugin instance.
    
    The category is a simple string - applications choose their own category names
    based on their needs (e.g., "transcription", "export", "data_processing", etc.).
    
    Example:
        ```python
        # Application defines its own categories
        PluginMetadata(
            name="whisper_transcriber",
            category="transcription",  # App-specific category
            title="Whisper Transcriber",
            config_schema={...}
        )
        ```
    
    Attributes:
        name: Internal plugin identifier
        category: Plugin category string (application-defined)
        title: Display title for the plugin
        config_schema: JSON Schema for plugin configuration
        description: Optional plugin description
        version: Optional plugin version
        is_configured: Whether the plugin has saved configuration
    """
    name: str
    category: str  # Application-defined category string
    title: str
    config_schema: Dict[str, Any]
    description: Optional[str] = None
    version: Optional[str] = None
    is_configured: bool = False
    
    def get_unique_id(self) -> str:
        """Generate unique ID for this plugin."""
        return self.name

## Plugin Registry Protocol

In [ ]:
#| export
@runtime_checkable
class PluginRegistryProtocol(Protocol):
    """Protocol that plugin registries should implement.
    
    This allows the settings library to work with any plugin system
    that implements these methods.
    """
    
    def get_plugin(self, unique_id: str) -> Optional[PluginMetadata]:
        """Get plugin metadata by unique ID."""
        ...
    
    def get_plugins_by_category(self, category: str) -> list[PluginMetadata]:
        """Get all plugins in a category."""
        ...
    
    def get_categories_with_plugins(self) -> list[str]:
        """Get all categories that have registered plugins."""
        ...
    
    def load_plugin_config(self, unique_id: str) -> Dict[str, Any]:
        """Load saved configuration for a plugin."""
        ...
    
    def save_plugin_config(self, unique_id: str, config: Dict[str, Any]) -> bool:
        """Save configuration for a plugin."""
        ...

## Simple Plugin Registry Implementation

In [ ]:
#| export
class SimplePluginRegistry:
    """Simple implementation of PluginRegistryProtocol.
    
    This provides a basic plugin registry that can be used with the settings
    library. Applications with more complex needs can implement their own
    registry that follows the PluginRegistryProtocol.
    
    Categories are arbitrary strings defined by the application.
    """
    
    def __init__(self, config_dir: Optional[Path] = None):
        self._plugins: Dict[str, PluginMetadata] = {}
        self._config_dir = config_dir or Path("configs")
    
    def register_plugin(self, metadata: PluginMetadata):
        """Register a plugin."""
        # Check if plugin is configured
        config_file = self._config_dir / f"{metadata.get_unique_id()}.json"
        metadata.is_configured = config_file.exists()
        
        self._plugins[metadata.get_unique_id()] = metadata
    
    def get_plugin(self, unique_id: str) -> Optional[PluginMetadata]:
        """Get plugin metadata by unique ID."""
        return self._plugins.get(unique_id)
    
    def get_plugins_by_category(self, category: str) -> list:
        """Get all plugins in a category."""
        return [p for p in self._plugins.values() if p.category == category]
    
    def get_categories_with_plugins(self) -> list:
        """Get all categories that have registered plugins."""
        categories = set(p.category for p in self._plugins.values())
        return sorted(categories)
    
    def load_plugin_config(self, unique_id: str) -> Dict[str, Any]:
        """Load saved configuration for a plugin."""
        import json
        config_file = self._config_dir / f"{unique_id}.json"
        if config_file.exists():
            with open(config_file, 'r') as f:
                return json.load(f)
        return {}
    
    def save_plugin_config(self, unique_id: str, config: Dict[str, Any]) -> bool:
        """Save configuration for a plugin."""
        import json
        try:
            self._config_dir.mkdir(exist_ok=True, parents=True)
            config_file = self._config_dir / f"{unique_id}.json"
            with open(config_file, 'w') as f:
                json.dump(config, f, indent=2)
            
            # Update is_configured status
            if unique_id in self._plugins:
                self._plugins[unique_id].is_configured = True
            
            return True
        except Exception as e:
            print(f"Error saving plugin config: {e}")
            return False

In [ ]:
# Example: Using SimplePluginRegistry with custom categories
import tempfile

with tempfile.TemporaryDirectory() as tmpdir:
    registry = SimplePluginRegistry(config_dir=Path(tmpdir))
    
    # Register plugins with application-specific categories
    registry.register_plugin(PluginMetadata(
        name="csv_exporter",
        category="export",  # Custom category
        title="CSV Exporter",
        config_schema={
            "type": "object",
            "properties": {
                "delimiter": {"type": "string", "default": ","}
            }
        },
        description="Export data to CSV format"
    ))
    
    registry.register_plugin(PluginMetadata(
        name="json_exporter",
        category="export",  # Same custom category
        title="JSON Exporter",
        config_schema={
            "type": "object",
            "properties": {
                "indent": {"type": "integer", "default": 2}
            }
        }
    ))
    
    registry.register_plugin(PluginMetadata(
        name="data_cleaner",
        category="preprocessing",  # Different custom category
        title="Data Cleaner",
        config_schema={
            "type": "object",
            "properties": {
                "remove_nulls": {"type": "boolean", "default": True}
            }
        }
    ))
    
    print(f"Registered plugins: {list(registry._plugins.keys())}")
    print(f"Categories: {registry.get_categories_with_plugins()}")
    print(f"Export plugins: {[p.name for p in registry.get_plugins_by_category('export')]}")
    print(f"Preprocessing plugins: {[p.name for p in registry.get_plugins_by_category('preprocessing')]}")
    
    # Save and load config
    registry.save_plugin_config("csv_exporter", {"delimiter": ";"})
    loaded = registry.load_plugin_config("csv_exporter")
    print(f"\nSaved and loaded config: {loaded}")
    print(f"Is configured: {registry.get_plugin('csv_exporter').is_configured}")

Registered plugins: ['csv_exporter', 'json_exporter', 'data_cleaner']
Categories: ['export', 'preprocessing']
Export plugins: ['csv_exporter', 'json_exporter']
Preprocessing plugins: ['data_cleaner']

Saved and loaded config: {'delimiter': ';'}
Is configured: True


In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()